建立 CSV 向量儲存

In [7]:
import os
import pandas as pd
import pymongo
import pprint
import certifi
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pymongo import MongoClient
import streamlit as st

# 設置環境變數
os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]
ATLAS_CONNECTION_STRING = st.secrets["MONGODB_URL"]

# 連接到 MongoDB Atlas
client = MongoClient(ATLAS_CONNECTION_STRING, tlsCAFile=certifi.where())
db_name = "TaiwanStockDatabase"
collection_name = "StockData"
atlas_collection = client[db_name][collection_name]
vector_search_index = "vector_index"

# 刪除現有數據的函數
def delete_existing_data():
    result = atlas_collection.delete_many({})
    return result.deleted_count

# 載入 CSV 資料
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

# 建立向量儲存
def initialize_data(file_path):
    # 刪除現有數據
    delete_existing_data()

    df = load_data(file_path)
    # 將每行數據拼接成一個描述字符串
    df["description"] = df.apply(
        lambda row: f"日期: {row['Date']}, 開盤: {row['Open']}, 最高: {row['High']}, 最低: {row['Low']}, 收盤: {row['Close']}, 調整後收盤: {row['Adj Close']}, 成交量: {row['Volume']}",
        axis=1,
    )
    documents = df["description"].tolist()
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(documents)
    vectors_list = vectors.toarray().tolist()

    # 建立向量儲存
    for doc, vector in zip(documents, vectors_list):
        atlas_collection.insert_one({"description": doc, "embedding": vector})
    
    # 儲存向量化器詞彙表
    with open("tfidf_vocabulary.json", "w") as f:
        json.dump(vectorizer.vocabulary_, f)

# 檢查集合是否為空，若為空則初始化資料
if atlas_collection.count_documents({}) == 0 or not os.path.exists("tfidf_vocabulary.json"):
    print("初始化資料並創建向量存儲...")
    initialize_data("台股大盤2015_2024.csv")
else:
    print("載入現有向量儲存...")

# 執行向量搜索的函數
def perform_vector_search(query):
    # 讀取向量化器詞彙表
    with open("tfidf_vocabulary.json", "r") as f:
        vocabulary = json.load(f)
    
    # 將查詢轉換為向量
    vectorizer = TfidfVectorizer(vocabulary=vocabulary)
    query_vector = vectorizer.fit_transform([query]).toarray()

    # 獲取所有已儲存的向量
    stored_vectors = list(
        atlas_collection.find({}, {"embedding": 1, "description": 1, "_id": 0})
    )
    descriptions = [v["description"] for v in stored_vectors]
    embeddings = [v["embedding"] for v in stored_vectors]

    # 計算餘弦相似度
    similarity_scores = cosine_similarity(query_vector, embeddings).flatten()
    sorted_indices = similarity_scores.argsort()[::-1]  # 按降序排列

    # 獲取前5個相似的文檔
    top_documents = [descriptions[i] for i in sorted_indices[:5]]
    return top_documents

# 問題
question = "請簡述2020年台股的市場表現"

# 獲取相關文檔
top_documents = perform_vector_search(question)

# 顯示結果
print("相關文檔：")
for doc in top_documents:
    print(doc)

# 可選顯示源文檔
print("查看源文檔：")
pprint.pprint(top_documents)


初始化資料並創建向量存儲...
相關文檔：
日期: 2024-05-24, 開盤: 21442.669922, 最高: 21608.720703, 最低: 21381.25, 收盤: 21565.339844, 調整後收盤: 21565.339844, 成交量: 4233200.0
日期: 2018-02-01, 開盤: 11139.400391, 最高: 11212.099609, 最低: 11139.400391, 收盤: 11160.25, 調整後收盤: 11160.25, 成交量: 2257600.0
日期: 2018-02-09, 開盤: 10371.009766, 最高: 10392.30957, 最低: 10189.040039, 收盤: 10371.75, 調整後收盤: 10371.75, 成交量: 2895400.0
日期: 2018-02-08, 開盤: 10559.44043, 最高: 10609.5, 最低: 10512.929688, 收盤: 10528.519531, 調整後收盤: 10528.519531, 成交量: 2287100.0
日期: 2018-02-07, 開盤: 10547.639648, 最高: 10697.820313, 最低: 10547.639648, 收盤: 10551.540039, 調整後收盤: 10551.540039, 成交量: 3145300.0
查看源文檔：
['日期: 2024-05-24, 開盤: 21442.669922, 最高: 21608.720703, 最低: 21381.25, 收盤: '
 '21565.339844, 調整後收盤: 21565.339844, 成交量: 4233200.0',
 '日期: 2018-02-01, 開盤: 11139.400391, 最高: 11212.099609, 最低: 11139.400391, 收盤: '
 '11160.25, 調整後收盤: 11160.25, 成交量: 2257600.0',
 '日期: 2018-02-09, 開盤: 10371.009766, 最高: 10392.30957, 最低: 10189.040039, 收盤: '
 '10371.75, 調整後收盤: 10371.75, 成交量: 2895400.0',
 '日

回答問題

In [8]:
import os
import pandas as pd
import pymongo
import pprint
import certifi
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pymongo import MongoClient
import streamlit as st

# 設置環境變數
os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]
ATLAS_CONNECTION_STRING = st.secrets["MONGODB_URL"]

# 連接到 MongoDB Atlas
client = MongoClient(ATLAS_CONNECTION_STRING, tlsCAFile=certifi.where())
db_name = "TaiwanStockDatabase"
collection_name = "StockData"
atlas_collection = client[db_name][collection_name]
vector_search_index = "vector_index"

# 刪除現有數據的函數
def delete_existing_data():
    result = atlas_collection.delete_many({})
    return result.deleted_count

# 載入 CSV 資料
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

# 建立向量儲存
def initialize_data(file_path):
    # 刪除現有數據
    delete_existing_data()

    df = load_data(file_path)
    # 將每行數據拼接成一個描述字符串
    df["description"] = df.apply(
        lambda row: f"日期: {row['Date']}, 開盤: {row['Open']}, 最高: {row['High']}, 最低: {row['Low']}, 收盤: {row['Close']}, 調整後收盤: {row['Adj Close']}, 成交量: {row['Volume']}",
        axis=1,
    )
    documents = df["description"].tolist()
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(documents)
    vectors_list = vectors.toarray().tolist()

    # 建立向量儲存
    for doc, vector in zip(documents, vectors_list):
        atlas_collection.insert_one({"description": doc, "embedding": vector})
    
    # 儲存向量化器詞彙表
    with open("tfidf_vocabulary.json", "w") as f:
        json.dump(vectorizer.vocabulary_, f)

# 檢查集合是否為空，若為空則初始化資料
if atlas_collection.count_documents({}) == 0 or not os.path.exists("tfidf_vocabulary.json"):
    print("初始化資料並創建向量存儲...")
    initialize_data("台股大盤2015_2024.csv")
else:
    print("載入現有向量儲存...")

# 執行向量搜索的函數
def perform_vector_search(query):
    # 讀取向量化器詞彙表
    with open("tfidf_vocabulary.json", "r") as f:
        vocabulary = json.load(f)
    
    # 將查詢轉換為向量
    vectorizer = TfidfVectorizer(vocabulary=vocabulary)
    query_vector = vectorizer.fit_transform([query]).toarray()

    # 調試信息：打印查詢向量的形狀
    print(f"Query vector shape: {query_vector.shape}")

    # 獲取所有已儲存的向量
    stored_vectors = list(
        atlas_collection.find({}, {"embedding": 1, "description": 1, "_id": 0})
    )
    descriptions = [v["description"] for v in stored_vectors]
    embeddings = [v["embedding"] for v in stored_vectors]

    # 調試信息：打印存儲向量的數量和每個向量的形狀
    print(f"Number of stored vectors: {len(embeddings)}")
    if embeddings:
        print(f"Shape of stored vectors: {len(embeddings[0])}")

    # 計算餘弦相似度
    similarity_scores = cosine_similarity(query_vector, embeddings).flatten()
    sorted_indices = similarity_scores.argsort()[::-1]  # 按降序排列

    # 獲取前5個相似的文檔
    top_documents = [descriptions[i] for i in sorted_indices[:5]]
    return top_documents

# 問題
question = "請簡述2020年台股的市場表現"

# 獲取相關文檔
top_documents = perform_vector_search(question)

# 顯示結果
print("回答：")
for doc in top_documents:
    print(doc)


載入現有向量儲存...
Query vector shape: (1, 7874)
Number of stored vectors: 2290
Shape of stored vectors: 7874
回答：
日期: 2024-05-24, 開盤: 21442.669922, 最高: 21608.720703, 最低: 21381.25, 收盤: 21565.339844, 調整後收盤: 21565.339844, 成交量: 4233200.0
日期: 2018-02-01, 開盤: 11139.400391, 最高: 11212.099609, 最低: 11139.400391, 收盤: 11160.25, 調整後收盤: 11160.25, 成交量: 2257600.0
日期: 2018-02-09, 開盤: 10371.009766, 最高: 10392.30957, 最低: 10189.040039, 收盤: 10371.75, 調整後收盤: 10371.75, 成交量: 2895400.0
日期: 2018-02-08, 開盤: 10559.44043, 最高: 10609.5, 最低: 10512.929688, 收盤: 10528.519531, 調整後收盤: 10528.519531, 成交量: 2287100.0
日期: 2018-02-07, 開盤: 10547.639648, 最高: 10697.820313, 最低: 10547.639648, 收盤: 10551.540039, 調整後收盤: 10551.540039, 成交量: 3145300.0


使用自然語言描述

In [15]:
import os
import pandas as pd
import pymongo
import pprint
import certifi
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pymongo import MongoClient
from langchain_openai import ChatOpenAI
import streamlit as st

# 設置環境變數
os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]
ATLAS_CONNECTION_STRING = st.secrets["MONGODB_URL"]

# 連接到 MongoDB Atlas
client = MongoClient(ATLAS_CONNECTION_STRING, tlsCAFile=certifi.where())
db_name = "TaiwanStockDatabase"
collection_name = "StockData"
atlas_collection = client[db_name][collection_name]
vector_search_index = "vector_index"

# 刪除現有數據的函數
def delete_existing_data():
    result = atlas_collection.delete_many({})
    return result.deleted_count

# 載入 CSV 資料
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

# 建立向量儲存
def initialize_data(file_path):
    # 刪除現有數據
    delete_existing_data()

    df = load_data(file_path)
    # 將每行數據拼接成一個描述字符串
    df["description"] = df.apply(
        lambda row: f"日期: {row['Date']}, 開盤: {row['Open']}, 最高: {row['High']}, 最低: {row['Low']}, 收盤: {row['Close']}, 調整後收盤: {row['Adj Close']}, 成交量: {row['Volume']}",
        axis=1,
    )
    documents = df["description"].tolist()
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(documents)
    vectors_list = vectors.toarray().tolist()

    # 建立向量儲存
    for doc, vector in zip(documents, vectors_list):
        atlas_collection.insert_one({"description": doc, "embedding": vector})
    
    # 儲存向量化器詞彙表
    with open("tfidf_vocabulary.json", "w") as f:
        json.dump(vectorizer.vocabulary_, f)

# 檢查集合是否為空，若為空則初始化資料
if atlas_collection.count_documents({}) == 0 or not os.path.exists("tfidf_vocabulary.json"):
    print("初始化資料並創建向量存儲...")
    initialize_data("台股大盤2015_2024.csv")
else:
    print("載入現有向量儲存...")

# 執行向量搜索的函數
def perform_vector_search(query, top_k=50):
    # 讀取向量化器詞彙表
    with open("tfidf_vocabulary.json", "r") as f:
        vocabulary = json.load(f)
    
    # 將查詢轉換為向量
    vectorizer = TfidfVectorizer(vocabulary=vocabulary)
    query_vector = vectorizer.fit_transform([query]).toarray()

    # 獲取所有已儲存的向量
    stored_vectors = list(
        atlas_collection.find({}, {"embedding": 1, "description": 1, "_id": 0})
    )
    descriptions = [v["description"] for v in stored_vectors]
    embeddings = [v["embedding"] for v in stored_vectors]

    # 計算餘弦相似度
    similarity_scores = cosine_similarity(query_vector, embeddings).flatten()
    sorted_indices = similarity_scores.argsort()[::-1]  # 按降序排列

    # 獲取前 top_k 個相似的文檔
    top_documents = [descriptions[i] for i in sorted_indices[:top_k]]
    return top_documents

# 使用OpenAI的模型來生成自然語言描述
def generate_answer(question, top_documents):
    llm = ChatOpenAI()
    context = "\n".join(top_documents)
    prompt = f"根據以下台股歷史數據回答問題：\n\n{context}\n\n問題：{question}\n\n回答："
    response = llm.invoke(prompt)
    return response.content.strip()

# 問題
question = "請簡述台股在近五年的市場表現差異。"

# 獲取相關文檔
top_documents = perform_vector_search(question)

# 生成回答
answer = generate_answer(question, top_documents)

# 顯示結果
print("回答：")
print(answer)


載入現有向量儲存...
回答：
根據提供的數據，我們可以看到在2018年2月至3月期間，台股的指數整體呈現下跌趨勢，並且波動幅度較大。而在2017年底至2018年初，台股指數則呈現較為穩定的表現。這顯示台股在近五年的市場表現存在較大的差異，且受到多個因素的影響，包括全球經濟環境、政治局勢、國際貿易關係等。投資者應該密切關注市場動態，制定合適的投資策略以應對市場波動。
